In [1]:
!pip install ain-py
!pip install nest_asyncio

In [2]:
import asyncio
import nest_asyncio
from time import time
from ain.ain import Ain
from ain.types import ValueOnlyTransactionInput

nest_asyncio.apply()

loop = asyncio.get_event_loop()

ain = Ain("https://testnet-api.ainetwork.ai", chainId=None)

# Create address and Set default account
accounts = ain.wallet.create(1)
my_address = accounts[0]
ain.wallet.setDefaultAccount(my_address)

# Use your own app name
app_name = f"my_app_{int(time()*1000)}"
app_path = f"/apps/{app_name}"

# Create an app at /apps/{appName}. With the admin config below,
# only 'my_address' will have owner & write permissions at /apps/{app_name}.
async def create_my_app():
    result = await ain.db.ref(
        f'/manager_app/{app_name}/create/{int(time()*1000)}'
    ).setValue(
        ValueOnlyTransactionInput(
            value={
                'admin': {
                    my_address: True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(create_my_app())

# print Owner of this App.
async def get_my_app_owner():
    result = await ain.db.ref(app_path).getOwner()
    print(result)
loop.run_until_complete(get_my_app_owner())


# Set write rule that allow everyone to set value.
async def set_write_rule():
    result = await ain.db.ref(app_path).setRule(
        ValueOnlyTransactionInput(
            value={
                '.rule': {
                    'write': True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(set_write_rule())

# Write value to user_message_path.
async def set_value():
    user_message_path = f'{app_path}/messages/{my_address}'
    result = await ain.db.ref(
        f'{user_message_path}/{int(time()*1000)}/user'
    ).setValue(
        ValueOnlyTransactionInput(value='Hello AI Network', nonce=-1)
    )
    print(result)
loop.run_until_complete(set_value())


{'tx_hash': '0x96db777def3e5c4102085ca14ff570191d590b2e6648ff3169e5a228d7f4cfcb', 'result': {'gas_amount_total': {'bandwidth': {'service': 1}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write permission on: /manager_app/my_app_1641549177683/create/1641549177684', 'code': 103, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 1}}
None
{'tx_hash': '0x03c676dafcb5c0113094c87fc908b75c704bbdc043d6daf7f699cc64c84ac89d', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641549177683': 1}}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write_rule permission on: /apps/my_app_1641549177683', 'code': 503, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 0}}
{'tx_hash': '0xac22cbdb5e0ae6791a38d75c1a61fc271945b7c330ffc6682f42393612743836', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641549177683': 1}}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write permission on: /app